In [ ]:
#####################################################################################################
##Copyright    O., Hamel et S. Lamari for The C00L07UN100120180002 Project.##########################
#####################################################################################################

#this file contains the code relating to the training of the Bi LSTM paraphrase generation model (English).

In [ ]:
# import the linguistic resources

%tensorflow_version 1.x
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


downloaded = drive.CreateFile({'id':'1pZjMamlLia5rRd0KEmLu-NnsIFBF3NKB'}) 
downloaded.GetContentFile('W2V.npy')

downloaded = drive.CreateFile({'id':'1OO6h6FH28KVRWMFJ25pA7dO_FH9t-THU'}) 
downloaded.GetContentFile('W2I.npy')

downloaded = drive.CreateFile({'id':'1YhtngRWwPlZoQ1IuRa_sGk_JXxfu9gZT'}) 
downloaded.GetContentFile('I2W.npy')

downloaded = drive.CreateFile({'id':'1eMsbuwJMnVBt9YPexoy58-pANJrhHddp'}) 
downloaded.GetContentFile('I2V.npy')

downloaded = drive.CreateFile({'id':'195SH3to94kQPZMkZadD4v04px_jkOmBN'}) 
downloaded.GetContentFile('questions.csv')

downloaded = drive.CreateFile({'id':'1-13mgnkRzDpIfT_f_-EK-WlyJsRBeckj'}) 
downloaded.GetContentFile('model.bin')

W2I,I2W,W2V,I2V = {},{},{},{}

data = np.load('W2V.npy', allow_pickle =True)
W2V.update(data.item()) 

data = np.load('W2I.npy', allow_pickle =True) 
W2I.update(data.item()) 

data =  np.load('I2W.npy', allow_pickle =True)
I2W.update(data.item())

data =  np.load('I2V.npy', allow_pickle =True)
I2V.update(data.item())

c = np.zeros(100)
I2V[0]=c
c=[]

downloaded = []

TensorFlow 1.x selected.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# import packages

import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

import sys

# define parameters

TRAIN_DATA_FILE = 'questions.csv'
EMBEDDING_FILE = 'model.bin'
MAX_SEQUENCE_LENGTH = 30
EMBEDDING_DIM = 100

# index word vectors

print('Indexing word vectors')

word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
        binary=True)
print('Found %s word vectors of word2vec' % len(word2vec.vocab))


# process texts in datasets

print('Processing text dataset')

# The function "text_to_wordlist" 
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)


texts_1 = [] 
texts_2 = []

with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        texts_1.append(text_to_wordlist(values[3]))
        texts_2.append(text_to_wordlist(values[4]))
print('Found %s texts in questions.csv' % len(texts_1))

# return only sentences that have a size less than or equal to 30, all of whose words are in the WE vocabulary


import nltk 
nltk.download('punkt')

t1, t2 = [], []
for i in range(len(texts_1)):

  wordList = nltk.word_tokenize(texts_1[i])
  test = True
  for e in wordList:
    test = test and e in word2vec.vocab
  if test == True and len(wordList) <= 30:
    t1.append(texts_1[i])
    t2.append(texts_2[i])

texts_1 = t1 
texts_2 = t2

t1, t2 = [], []
for i in range(len(texts_2)):
  wordList = nltk.word_tokenize(texts_2[i])
  test = True
  for e in wordList:
    test = test and e in word2vec.vocab
  if test == True and len(wordList) <= 30:
    t1.append(texts_1[i])
    t2.append(texts_2[i])

texts_1 = t1 
texts_2 = t2 

print('Found %s texts in questions.csv' % len(texts_1))



tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts_1 + texts_2)

sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)


word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH,padding='post')
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH,padding='post')

print('Shape of data tensor:', data_1.shape)

Using TensorFlow backend.


Indexing word vectors


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Found 4027169 word vectors of word2vec
Processing text dataset
Found 404351 texts in questions.csv
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Found 383898 texts in questions.csv
Found 72397 unique tokens
Shape of data tensor: (383898, 30)


In [ ]:
# import data science packages
import re
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential,Model
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Dense, Activation, Dropout, Embedding, Flatten, Bidirectional, Input, LSTM
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy, mean_squared_error, mean_absolute_error, logcosh
from keras.layers.normalization import BatchNormalization
from keras import regularizers
 
from keras.utils import to_categorical 



import math
import random
import sys
import matplotlib.pyplot as plt
from keras.utils import plot_model

num_train_patters = len(data_1)

print('Total patterns:\t' + str(num_train_patters))

Total patterns:	383898


In [ ]:
#define parameters

rnn_size = 256 # size of RNN
learning_rate = 0.0001 #learning rate
from keras.callbacks import EarlyStopping,ModelCheckpoint
import tensorflow as tf

#create the model

print('Building LSTM model...')
model = Sequential()
model.add(Bidirectional(LSTM(rnn_size, activation="relu",return_sequences = True),input_shape=(30, 100)))
model.add(Dropout(0.5))
model.add(Dense(len(I2V),activation="softmax"))
optimizer = Adam(lr=learning_rate)
loss_function_used = 'categorical_crossentropy'
callbacks=[EarlyStopping(patience=2, monitor='val_acc')]
model.compile(optimizer='Adam',loss= loss_function_used,metrics=['acc'])
print('LSTM model built.')

Building LSTM model...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
LSTM model built.


In [ ]:
# data partition part

import numpy as np
import keras
import csv
from tensorflow.python.keras.utils.data_utils import Sequence
from tensorflow import keras
import numpy as np
from gensim.models import KeyedVectors
from gensim.models import Word2Vec


class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1, n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        global data_2
        global data_1
        self.x_data,self.y_data= data_1, data_2
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))


    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):

        global I2V

        X = np.zeros((self.batch_size, 30,100),'float64')
        Y = np.zeros((self.batch_size, 30,len(I2V)),'float64')
        

        #for i, ID in enumerate(list_IDs_temp):
        for i,ID in enumerate(list_IDs_temp) :
            sentence = self.x_data[ID]
            paraphrase = self.y_data[ID]
            pos = 0
            for j in sentence:
                X[i, pos,] = I2V[j]
                pos = pos+1
            pos = 0
            for j in paraphrase:
                Y[i, pos,j] = 1
                pos = pos+1

                
        return X, Y


In [ ]:
#Train part

import keras 

params = {'dim': (32,32,32),
          'batch_size': 32,
          'n_channels': 1,
          'n_classes': 6,
          'shuffle': True
          }

# Datasets

trainingSet = [i for i in range(0,307118)]
validationSet = trainingSet[268728:307118]
trainingSet = trainingSet[0:268728]


partition = {}
partition['train'] = trainingSet
partition['validation'] = validationSet


# Generators
training_generator = DataGenerator(partition['train'], data_2, **params)
validation_generator = DataGenerator(partition['validation'], data_2, **params)


"""

recuperate the model in order to continue training

model = keras.models.load_model('/content/drive/My Drive/English DataSet 1sens/model.h5')

"""

print(model.summary())

acc  = float('-inf')

print(model.summary())

#start the train

for i in range (500):

  history = model.fit_generator(generator=training_generator,validation_data=validation_generator,epochs = 1,use_multiprocessing=True,workers=1)
  
    
  #recuperation of loss & acc
  curr_loss = history.history['loss'][0]
  curr_acc = history.history['acc'][0]
  curr_val_acc = history.history['val_accuracy'][0] 
  curr_val_loss = history.history['val_loss'][0]

  loss_file=open("/content/drive/My Drive/English DataSet 1sens/lossOneHotBiLSTM.txt", "a+")
  acc_file=open("/content/drive/My Drive/English DataSet 1sens/accOneHotBiLSTM.txt", "a+")
  loss_file.write('\n'+str(curr_loss))
  acc_file.write('\n'+str(curr_acc))
  
  loss_file.close()
  acc_file.close()

  #saving model

  if curr_acc > acc :
    model.save('/content/drive/My Drive/English DataSet 1sens/bestmodel.h5')
    acc = curr_acc
  else:
    model.save('/content/drive/My Drive/English DataSet 1sens/model.h5')
  

Epoch 1/1
8396/8397 [============================>.] - ETA: 1s - loss: 1.1921e-07 - acc: 0.6392

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 19 could not be retrieved. It could be because a worker has died.
  UserWarning)
